# Fine tuning classification example

We will fine-tune a `babbage-002` classifier (replacement for the `ada` models) to distinguish between the two sports: Baseball and Hockey.

In [20]:
import dotenv
import openai
from sklearn.datasets import fetch_20newsgroups

dotenv.load_dotenv(".env")

client = openai.OpenAI()

In [21]:
# Dirty patch to fix SSL error
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context
    
categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

 ## Data exploration
 The newsgroup dataset can be loaded using sklearn. First we will look at the data itself:

In [22]:
print(sports_dataset['data'][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [24]:
# The target is a number, we can get the name of the category using the target_names attribute
sports_dataset.target_names[sports_dataset['target'][0]]


'rec.sport.baseball'

In [26]:
# some basic statistics
len_all = len(sports_dataset.data)
len_baseball = len([e for e in sports_dataset.target if e == 0])
len_hockey = len([e for e in sports_dataset.target if e == 1])

print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

Total examples: 1197, Baseball examples: 597, Hockey examples: 600


One sample from the baseball category can be seen above. It is an email to a mailing list. We can observe that we have 1197 examples in total, which are evenly split between the two sports.

## Data Preparation
We transform the dataset into a pandas dataframe, with a column for prompt and completion. The prompt contains the email from the mailing list, and the completion is a name of the sport, either hockey or baseball. For demonstration purposes only and speed of fine-tuning we take only 300 examples. In a real use case the more examples the better the performance.

In [61]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
texts = [text.replace('\n', ' ').replace('\t', ' ') for text in texts]
texts = [text[:5000] for text in texts]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()
# df.iloc[325]
# get the 325 row
# len(df.iloc[325]['prompt'])

# find the biggest len of the prompt
# max_len = 0
# index = 0
# for i in range(len(df)):
#     if len(df.iloc[i]['prompt']) > max_len:
#         max_len = len(df.iloc[i]['prompt'])
#         index = i
# max_len
# index

,prompt,completion
0,From: dougb@comm.mot.com (Doug Bank) Subject: ...,baseball
1,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
2,From: rudy@netcom.com (Rudy Wade) Subject: Re:...,baseball
3,From: monack@helium.gas.uug.arizona.edu (david...,hockey
4,Subject: Let it be Known From: <ISSBTL@BYUVM.B...,baseball


Both baseball and hockey are single tokens. We save the dataset as a jsonl file.

In [62]:
df.to_json("data/classification/sport2.jsonl", orient='records', lines=True)

### Data Preparation tool
We can now use a data preparation tool which will suggest a few improvements to our dataset before fine-tuning. Before launching the tool we update the openai library to ensure we're using the latest data preparation tool. We additionally specify `-q` which auto-accepts all suggestions.

In [63]:
# we need to delete the old files first
try:
    os.remove("data/classification/sport2_prepared_train.jsonl")
    os.remove("data/classification/sport2_prepared_valid.jsonl")
except FileNotFoundError:
    pass

In [64]:
!openai tools fine_tunes.prepare_data -f data/classification/sport2.jsonl -q

Analyzing...

- Your file contains 1197 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for

The tool helpfully suggests a few improvements to the dataset and splits the dataset into training and validation set.

A suffix between a prompt and a completion is necessary to tell the model that the input text has stopped, and that it now needs to predict the class. Since we use the same separator in each example, the model is able to learn that it is meant to predict either baseball or hockey following the separator.
A whitespace prefix in completions is useful, as most word tokens are tokenized with a space prefix.
The tool also recognized that this is likely a classification task, so it suggested to split the dataset into training and validation datasets. This will allow us to easily measure expected performance on new data.

## Fine-tuning
The tool suggests we run the following command to train the dataset. Since this is a classification task, we would like to know what the generalization performance on the provided validation set is for our classification use case.

We can simply copy the suggested command from the CLI tool. We specifically add `-m ada` to fine-tune a cheaper and faster ada model, which is usually comperable in performance to slower and more expensive models on classification use cases. 

In [65]:
train_file = client.files.create(file=open("data/classification/sport2_prepared_train.jsonl", "rb"), purpose="fine-tune")
valid_file = client.files.create(file=open("data/classification/sport2_prepared_valid.jsonl", "rb"), purpose="fine-tune")

fine_tuning_job = client.fine_tuning.jobs.create(training_file=train_file.id, validation_file=valid_file.id, model="babbage-002")

print(fine_tuning_job)

FineTuningJob(id='ftjob-9X0GzrZdpUPxDuEVhSInGFR5', created_at=1729084872, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-SKY9DMIGk3y6f5KAypuz1L4A', result_files=[], seed=855664309, status='validating_files', trained_tokens=None, training_file='file-ziO1h3phyZ9kxqYgZDfPpa8I', validation_file='file-RZRnZHLdyKuWg6pslzUzWnuQ', estimated_finish=None, integrations=[], user_provided_suffix=None)


The model is successfully trained in about ten minutes. You can watch the finetune happen on [https://platform.openai.com/finetune/](https://platform.openai.com/finetune/)

You can also check on its status programatically:

In [86]:
fine_tune_results = client.fine_tuning.jobs.retrieve(fine_tuning_job.id)
fine_tune_results

FineTuningJob(id='ftjob-9X0GzrZdpUPxDuEVhSInGFR5', created_at=1729084872, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:babbage-002:sloboda-studio-com::AIySppiQ', finished_at=1729085689, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=16), model='babbage-002', object='fine_tuning.job', organization_id='org-SKY9DMIGk3y6f5KAypuz1L4A', result_files=['file-3l8Xs2eY9CuzBR30g3D2B4CT'], seed=855664309, status='succeeded', trained_tokens=1204935, training_file='file-ziO1h3phyZ9kxqYgZDfPpa8I', validation_file='file-RZRnZHLdyKuWg6pslzUzWnuQ', estimated_finish=None, integrations=[], user_provided_suffix=None)

## Using the model
We can now call the model to get the predictions.

In [90]:
test = pd.read_json('data/classification/sport2_prepared_valid.jsonl', lines=True)
test.head()

,prompt,completion
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
1,From: smorris@venus.lerc.nasa.gov (Ron Morris ...,hockey
2,From: golchowy@alchemy.chem.utoronto.ca (Geral...,hockey
3,From: krattige@hpcc01.corp.hp.com (Kim Krattig...,baseball
4,From: warped@cs.montana.edu (Doug Dolven) Subj...,baseball


We need to use the same separator following the prompt which we used during fine-tuning. In this case it is `\n\n###\n\n`. Since we're concerned with classification, we want the temperature to be as low as possible, and we only require one token completion to determine the prediction of the model.

In [97]:
ft_model = fine_tune_results.fine_tuned_model

# note that this calls the legacy completions api - https://platform.openai.com/docs/api-reference/completions
res = client.completions.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0)
print(test["prompt"][0])
res.choices[0].text


From: gld@cunixb.cc.columbia.edu (Gary L Dare) Subject: Re: Flames Truly Brutal in Loss Nntp-Posting-Host: cunixb.cc.columbia.edu Reply-To: gld@cunixb.cc.columbia.edu (Gary L Dare) Organization: PhDs In The Hall Distribution: na Lines: 13   This game would have been great as part of a double-header on ABC or ESPN; the league would have been able to push back-to-back wins by Le Magnifique and The Great One.  Unfortunately, the only network that would have done that was SCA, seen in few areas and hard to justify as a pay channel. )-;  gld -- ~~~~~~~~~~~~~~~~~~~~~~~~ Je me souviens ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Gary L. Dare > gld@columbia.EDU    GO  Winnipeg Jets  GO!!! > gld@cunixc.BITNET   Selanne + Domi ==> Stanley

###




' hockey'

To get the log probabilities, we can specify logprobs parameter on the completion request

In [92]:
res = client.completions.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].logprobs.top_logprobs

[{' hockey': 0.0, '1': -21.813498}]

We can see that the model predicts hockey as a lot more likely than baseball, which is the correct prediction. By requesting log_probs, we can see the prediction (log) probability for each class.

### Generalization
Interestingly, our fine-tuned classifier is quite versatile. Despite being trained on emails to different mailing lists, it also successfully predicts tweets.

In [96]:
sample_hockey_tweet = """
Are the Maple Leafs playing boring? What stands out with the hot Calgary Flames? Will there be another elite French Canadian NHL goalie after Fleury? All that and more.
"""
res = client.completions.create(model=ft_model, prompt=sample_hockey_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].text

' hockey'

In [94]:
sample_baseball_tweet="""BREAKING: The Tampa Bay Rays are finalizing a deal to acquire slugger Nelson Cruz from the Minnesota Twins, sources tell ESPN."""
res = client.completions.create(model=ft_model, prompt=sample_baseball_tweet + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res.choices[0].text

' baseball'